In [1]:
import sqlalchemy as sa
import urllib
import pandas as pd
import pickle5 as pickle
from tqdm import tqdm
import dask.dataframe as dd
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Słowniki SmogoliczkaArchives
metadane_STANOWISKA = pd.read_excel(
    "./dictionaries/Metadane - stacje i stanowiska pomiarowe.xlsx", sheet_name='STANOWISKA')
metadane_STANOWISKA.head(5)

,Nr,KodStanowiska,KodStacji,NazwaStacji,StaryKodStacji,WskaznikKod,Wskaznik,CzasUsredniania,TypPomiaru,DataUruchomienia,DataZamkniecia,Wojewodztwo,NazwaStrefy
0,1,DsBialka-CO-1g,DsBialka,Białka,DsBialka,CO,tlenek węgla,1-godzinny,automatyczny,2004-01-01,2005-12-31,DOLNOŚLĄSKIE,strefa dolnośląska
1,2,DsBialka-PM10-1g,DsBialka,Białka,DsBialka,PM10,pył zawieszony PM10,1-godzinny,automatyczny,2004-01-01,2005-12-31,DOLNOŚLĄSKIE,strefa dolnośląska
2,3,DsBialka-SO2-1g,DsBialka,Białka,DsBialka,SO2,dwutlenek siarki,1-godzinny,automatyczny,2004-01-01,2005-12-31,DOLNOŚLĄSKIE,strefa dolnośląska
3,4,DsBialka-PM10-24g,DsBialka,Białka,DsBialka,PM10,pył zawieszony PM10,24-godzinny,manualny,2004-01-01,2005-12-31,DOLNOŚLĄSKIE,strefa dolnośląska
4,5,DsBialka-Pb(PM10)-24g,DsBialka,Białka,DsBialka,Pb(PM10),ołów w PM10,24-godzinny,manualny,1990-01-03,2005-12-31,DOLNOŚLĄSKIE,strefa dolnośląska


In [3]:
metadane_STACJE=pd.read_excel("./dictionaries/Metadane - stacje i stanowiska pomiarowe.xlsx",sheet_name='STACJE')
metadane_STACJE.head(5)

,Nr,KodStacji,KodMiedzynarodowy,NazwaStacji,StaryKodStacji,DataUruchomienia,DataZamkniecia,TypStacji,TypObszaru,RodzajStacji,Wojewodztwo,Miejscowosc,Ulica,Latitude,Longitude
0,1,DsBialka,NaN,Białka,DsBialka,1990-01-03,2005-12-31,przemysłowa,podmiejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Białka,NaN,51.197783,16.117390
1,2,DsBielGrot,NaN,Bielawa - ul. Grota Roweckiego,DsBielGrot,1994-01-02,2003-12-31,tło,miejski,w budynku,DOLNOŚLĄSKIE,Bielawa,ul. Grota Roweckiego 6,50.682510,16.617348
2,3,DsBogatFrancMOB,PL0602A,Bogatynia Mobil,DsBogatMob,2015-01-01,2015-12-31,tło,miejski,mobilna,DOLNOŚLĄSKIE,Bogatynia,ul. Francuska/Kręta,50.940998,14.916790
3,4,DsBogChop,PL0315A,Bogatynia - Chopina,DsBogChop,1996-01-01,2013-12-31,przemysłowa,miejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Bogatynia,ul. Chopina 35,50.905856,14.967175
4,5,DsBogZatonieMob,PL0576A,Bogatynia - Mobil,DsBogZatonieMob,2012-01-01,2012-12-31,przemysłowa,miejski,mobilna,DOLNOŚLĄSKIE,Bogatynia,"ul. Konrada, Zatonie",50.943245,14.913327


In [4]:
#Wczytujemy słownik StacjePomiarowe - sql
StacjePomiarowe_dict=pd.read_csv("./dictionaries/stacjeSmogoliczka.csv",sep=';')
StacjePomiarowe_dict.head()

,idstacji,NazwaStacji,SzerGeo,DlugGeo,IdMiasta,Miasto,Gmina,Powiat,Wojewodztwo,lokalizacja
0,11,Czerniawa,50.912475,15.312190,142,Czerniawa,Świeradów-Zdrój,lubański,DOLNOŚLĄSKIE,0xE6100000010CC746205ED79F2E4074B515FBCB744940
1,14,Działoszyn,50.972167,14.941319,192,Działoszyn,Bogatynia,zgorzelecki,DOLNOŚLĄSKIE,0xE6100000010CF4346090F4E12D400EA0DFF76F7C4940
2,16,Dzierżoniów - Piłsudskiego,50.732817,16.648050,198,Dzierżoniów,Dzierżoniów,dzierżoniowski,DOLNOŚLĄSKIE,0xE6100000010C3D2CD49AE6A53040F6798CF2CC5D4940
3,38,Kłodzko - Szkolna,50.433493,16.653660,368,Kłodzko,Kłodzko,kłodzki,DOLNOŚLĄSKIE,0xE6100000010C0FB4024356A73040C005D9B27C374940
4,52,Legnica - Rzeczypospolitej,51.204503,16.180513,453,Legnica,Legnica,Legnica,DOLNOŚLĄSKIE,0xE6100000010CBB809719362E30408A7780272D9A4940


In [5]:
df_sql = pd.merge(StacjePomiarowe_dict, metadane_STACJE, left_on=['NazwaStacji'], right_on=['NazwaStacji'], how='left')
df_sql.head()

,idstacji,NazwaStacji,SzerGeo,DlugGeo,IdMiasta,Miasto,Gmina,Powiat,Wojewodztwo_x,lokalizacja,Nr,KodStacji,KodMiedzynarodowy,StaryKodStacji,DataUruchomienia,DataZamkniecia,TypStacji,TypObszaru,RodzajStacji,Wojewodztwo_y,Miejscowosc,Ulica,Latitude,Longitude
0,11,Czerniawa,50.912475,15.312190,142,Czerniawa,Świeradów-Zdrój,lubański,DOLNOŚLĄSKIE,0xE6100000010CC746205ED79F2E4074B515FBCB744940,10.0,DsCzerStraza,PL0028A,DsCzer02,1996-07-01,NaT,tło,pozamiejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Czerniawa,ul. Strażacka 7,50.912475,15.312190
1,14,Działoszyn,50.972167,14.941319,192,Działoszyn,Bogatynia,zgorzelecki,DOLNOŚLĄSKIE,0xE6100000010CF4346090F4E12D400EA0DFF76F7C4940,14.0,DsDzialoszyn,PL0054A,DsDzia01,1996-07-01,NaT,przemysłowa,pozamiejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Działoszyn,bez ulicy,50.972167,14.941319
2,16,Dzierżoniów - Piłsudskiego,50.732817,16.648050,198,Dzierżoniów,Dzierżoniów,dzierżoniowski,DOLNOŚLĄSKIE,0xE6100000010C3D2CD49AE6A53040F6798CF2CC5D4940,15.0,DsDziePilsud,PL0187A,DsDzierPilsA,2005-01-01,NaT,tło,miejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Dzierżoniów,ul. Piłsudskiego 26,50.732817,16.648050
3,38,Kłodzko - Szkolna,50.433493,16.653660,368,Kłodzko,Kłodzko,kłodzki,DOLNOŚLĄSKIE,0xE6100000010C0FB4024356A73040C005D9B27C374940,44.0,DsKlodzSzkol,PL0504A,DsKlodzkoA,2009-06-05,NaT,tło,miejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Kłodzko,ul. Szkolna 8,50.433493,16.653660
4,52,Legnica - Rzeczypospolitej,51.204503,16.180513,453,Legnica,Legnica,Legnica,DOLNOŚLĄSKIE,0xE6100000010CBB809719362E30408A7780272D9A4940,53.0,DsLegAlRzecz,PL0190A,DsLegRzeczA,2004-10-01,NaT,tło,miejski,kontenerowa stacjonarna,DOLNOŚLĄSKIE,Legnica,al. Rzeczypospolitej 10/12,51.204503,16.180513


In [6]:
df_sql.shape

(243, 24)

In [7]:
df_sql=df_sql[['NazwaStacji','KodStacji']]

In [8]:
len(df_sql[df_sql['KodStacji'].isnull()])

43

In [9]:
test=df_sql[df_sql['KodStacji'].isnull()]
test['NazwaStacji'].to_list()

['Żywiec, ul. Kopernika  83 a',
 'Chełmno, Łunawska 3A',
 'Nakło nad Notecią, Park 700-lecia',
 'Grajewo - mobilne ',
 'Lubań',
 'Augustów - mobilne ',
 'Środa Śląska',
 'Kamienna Góra',
 'Ostrowiec Świętokrzyski, MOBILNA',
 'Suwałki, ul. Pułaskiego',
 'Skawina, ul. Studzińskiego',
 'Latoszyn-Zdrój-mobilna',
 'NałęczówMOB',
 'WIOŚ Biskupiec-Mobilna',
 'Goczałkowice Zdrój, ul. Parkowa',
 'CHojnice - mobilna',
 'Łask-Narutowicza 28',
 'Kutno-1 Maja 7',
 'Gołuchów Ujęcie Wody',
 'Racibórz, Wojska Polskiego 8',
 'Augustów-Miejska',
 'Trzebnica',
 'Jędrzejów, MOBILNA',
 'Sandomierz, MOBILNA',
 'Mszana Dolna',
 'Solec Kujawski, ul. gen. Stefana Roweckiego "Grota"',
 'Polanczyk-mobilna',
 'Milicz',
 'Krasnobrod',
 'Opatów, MOBILNA',
 'Uzdrowisko_Połczyn-Zdrój',
 'Strzelce Opolskie mobilna',
 'Kielce, ul. Warszawska, komunikacyjna',
 'Świebodzin,ul. Gen.W.Sikorskiego',
 'Sieprc-WiosnyLudówZSznr2',
 'Kołobrzeg_KasprowiczaMOB',
 'Cieszyn, ul. Chopina 37',
 'Gubin, ul. Gen. W. Andersa',
 'Brzeg m

In [11]:
ddict = {'Suwałki, ul. Pułaskiego':'Suwałki, ul. Pułaskiego 26',
         'Żywiec, ul. Kopernika  83 a':'Żywiec, ul. Kopernika 83 a',
         'Augustów - mobilne ': 'Augustów - mobilne'
        }
print(ddict)

{'Suwałki, ul. Pułaskiego': 'Suwałki, ul. Pułaskiego 26', 'Żywiec, ul. Kopernika  83 a': 'Żywiec, ul. Kopernika 83 a', 'Augustów - mobilne ': 'Augustów - mobilne'}


In [12]:
StacjePomiarowe_dict.replace({"NazwaStacji":ddict}, inplace=True)

In [13]:
df_sqlv2 = pd.merge(StacjePomiarowe_dict, metadane_STACJE, left_on=['NazwaStacji'], right_on=['NazwaStacji'], how='left')


In [14]:
len(df_sqlv2[df_sqlv2['KodStacji'].isnull()])

40

In [15]:
test2=df_sqlv2[df_sqlv2['KodStacji'].isnull()]
test2['NazwaStacji'].to_list()

['Chełmno, Łunawska 3A',
 'Nakło nad Notecią, Park 700-lecia',
 'Grajewo - mobilne ',
 'Lubań',
 'Środa Śląska',
 'Kamienna Góra',
 'Ostrowiec Świętokrzyski, MOBILNA',
 'Skawina, ul. Studzińskiego',
 'Latoszyn-Zdrój-mobilna',
 'NałęczówMOB',
 'WIOŚ Biskupiec-Mobilna',
 'Goczałkowice Zdrój, ul. Parkowa',
 'CHojnice - mobilna',
 'Łask-Narutowicza 28',
 'Kutno-1 Maja 7',
 'Gołuchów Ujęcie Wody',
 'Racibórz, Wojska Polskiego 8',
 'Augustów-Miejska',
 'Trzebnica',
 'Jędrzejów, MOBILNA',
 'Sandomierz, MOBILNA',
 'Mszana Dolna',
 'Solec Kujawski, ul. gen. Stefana Roweckiego "Grota"',
 'Polanczyk-mobilna',
 'Milicz',
 'Krasnobrod',
 'Opatów, MOBILNA',
 'Uzdrowisko_Połczyn-Zdrój',
 'Strzelce Opolskie mobilna',
 'Kielce, ul. Warszawska, komunikacyjna',
 'Świebodzin,ul. Gen.W.Sikorskiego',
 'Sieprc-WiosnyLudówZSznr2',
 'Kołobrzeg_KasprowiczaMOB',
 'Cieszyn, ul. Chopina 37',
 'Gubin, ul. Gen. W. Andersa',
 'Brzeg mobilna',
 'Mosina, ul. Czereśniowa',
 'Kielce, ul. Jurajska',
 'Warszawa-IMiGW-PIB',

In [ ]:
df_sqlv2_test=df_sqlv2[['idstacji','NazwaStacji','KodStacji']]

In [18]:
df_sqlv2_test.head()

,idstacji,NazwaStacji,KodStacji
0,11,Czerniawa,DsCzerStraza
1,14,Działoszyn,DsDzialoszyn
2,16,Dzierżoniów - Piłsudskiego,DsDziePilsud
3,38,Kłodzko - Szkolna,DsKlodzSzkol
4,52,Legnica - Rzeczypospolitej,DsLegAlRzecz


In [19]:
SensoryPomiarowe=pd.read_csv("./dictionaries/SensoryPomiarowe.csv",sep=';')

In [7]:
SensoryPomiarowe.head()

,IdSensora,IdStacji,NazwaParametru,ParFormula,ParKod,IdParametru
0,50,11,dwutlenek azotu,NO2,NO2,6
1,52,11,ozon,O3,O3,5
2,56,11,dwutlenek siarki,SO2,SO2,1
3,88,14,dwutlenek azotu,NO2,NO2,6
4,92,14,pył zawieszony PM10,PM10,PM10,3


In [24]:
SensoryPomiarowe.ParKod.unique()

array(['NO2', 'O3', 'SO2', 'PM10', 'C6H6', 'CO', 'PM2.5'], dtype=object)

In [42]:
df_sqlv3 = pd.merge(df_sqlv2_test, SensoryPomiarowe, left_on=['idstacji'], right_on=['IdStacji'], how='left')

In [43]:
df_sqlv3.head()

,idstacji,NazwaStacji,KodStacji,IdSensora,IdStacji,NazwaParametru,ParFormula,ParKod,IdParametru
0,11,Czerniawa,DsCzerStraza,50.0,11.0,dwutlenek azotu,NO2,NO2,6.0
1,11,Czerniawa,DsCzerStraza,52.0,11.0,ozon,O3,O3,5.0
2,11,Czerniawa,DsCzerStraza,56.0,11.0,dwutlenek siarki,SO2,SO2,1.0
3,14,Działoszyn,DsDzialoszyn,88.0,14.0,dwutlenek azotu,NO2,NO2,6.0
4,14,Działoszyn,DsDzialoszyn,92.0,14.0,pył zawieszony PM10,PM10,PM10,3.0


In [44]:
df_sqlv3.drop(columns=['idstacji','NazwaParametru','ParFormula','IdSensora'],inplace=True)

In [45]:
df_sqlv3.head()

,NazwaStacji,KodStacji,IdStacji,ParKod,IdParametru
0,Czerniawa,DsCzerStraza,11.0,NO2,6.0
1,Czerniawa,DsCzerStraza,11.0,O3,5.0
2,Czerniawa,DsCzerStraza,11.0,SO2,1.0
3,Działoszyn,DsDzialoszyn,14.0,NO2,6.0
4,Działoszyn,DsDzialoszyn,14.0,PM10,3.0


In [91]:
%time


df = dd.read_csv('./raw_data/filtered/smogoliczka24.01.csv',sep=';',decimal=',').compute()
df["Czas pomiaru"]=pd.to_datetime(df.Data) + df.Godzina.astype('timedelta64[h]')
df.drop(columns=['Data','Godzina','idsensora','status','Klucz','idpomiaru'],inplace=True,axis=0)
mask= df['Czas pomiaru']>'2020-01-01 00:00:00'
df=df[mask]
df.reset_index(drop=True, inplace=True)
df.head(5)

Wall time: 0 ns
[########################################] | 100% Completed |  5.7s
[########################################] | 100% Completed |  5.8s
[########################################] | 100% Completed |  5.9s
[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.1s
[########################################] | 100% Completed |  6.2s


,dane,IDRodzaju,idstacji,Czas pomiaru
0,1.9269,6,14,2020-01-01 01:00:00
1,6.4941,1,14,2020-01-01 01:00:00
2,4.9229,6,16,2020-01-01 01:00:00
3,3.4370,1,16,2020-01-01 01:00:00
4,18.9957,6,38,2020-01-01 01:00:00


In [92]:
df_smogapi1 = pd.merge(df, df_sqlv3, left_on=['IDRodzaju','idstacji'], right_on=['IdParametru','IdStacji'], how='left')
df_smogapi1=df_smogapi1[['KodStacji','Czas pomiaru','dane','ParKod']]
df_smogapi1.rename(columns={"KodStacji": "Kod stacji_metadane","ParKod":"Wskaźnik"}, inplace=True)


In [93]:
df_smogapi1.head()

,Kod stacji_metadane,Czas pomiaru,dane,Wskaźnik
0,DsDzialoszyn,2020-01-01 01:00:00,1.9269,NO2
1,DsDzialoszyn,2020-01-01 01:00:00,6.4941,SO2
2,DsDziePilsud,2020-01-01 01:00:00,4.9229,NO2
3,DsDziePilsud,2020-01-01 01:00:00,3.4370,SO2
4,DsKlodzSzkol,2020-01-01 01:00:00,18.9957,NO2


In [94]:
ddf = dd.from_pandas(df_smogapi1,npartitions=1)

In [95]:
def f(df: pd.DataFrame) -> pd.DataFrame:
    """ This is your code from above, as a function """
    return df.pivot_table(index=['Kod stacji_metadane','Czas pomiaru'], columns='Wskaźnik',values='dane').reset_index()

columns = ["Kod stacji_metadane","Czas pomiaru","C6H6","CO","NO2","O3","PM10","PM2.5","SO2"]
df = pd.DataFrame(columns=columns)
df.head(0)

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2


In [96]:
from dask.diagnostics import ProgressBar
ProgressBar().register()
ddf=ddf.map_partitions(f,meta=df)  # apply to all pandas dataframes within dask dataframe


In [97]:
ddf=ddf.repartition(npartitions= 10)

In [98]:
ddf.head()

[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.1s
[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.4s
[########################################] | 100% Completed |  6.5s
[########################################] | 100% Completed |  6.5s


Wskaźnik,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2
0,DsCzerStraza,2020-06-09 13:00:00,NaN,NaN,NaN,79.2056,NaN,NaN,NaN
1,DsCzerStraza,2020-06-09 14:00:00,NaN,NaN,1.6385,84.7211,NaN,NaN,NaN
2,DsCzerStraza,2020-06-09 15:00:00,NaN,NaN,1.6974,89.3183,NaN,NaN,NaN
3,DsCzerStraza,2020-06-09 16:00:00,NaN,NaN,1.9131,93.0739,NaN,NaN,NaN
4,DsCzerStraza,2020-06-09 17:00:00,NaN,NaN,1.9895,91.5978,NaN,NaN,NaN


In [101]:
from loader_sql import PandasToSQL
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

SERVER = os.environ.get("SERVER")
DATABASE = os.environ.get("DATABASE")

USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")

In [102]:
PandasToSQL().dask_loader(ddf,"pomiary_pivoted")

  0%|          | 0/10 [00:00<?, ?it/s]

[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.4s
[########################################] | 100% Completed |  6.5s
[########################################] | 100% Completed |  6.6s
[########################################] | 100% Completed |  6.7s
[########################################] | 100% Completed |  6.8s


 10%|█         | 1/10 [00:35<05:20, 35.59s/it]

[########################################] | 100% Completed |  5.8s
[########################################] | 100% Completed |  5.9s
[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.1s
[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.4s


 20%|██        | 2/10 [01:10<04:43, 35.47s/it]

...

 80%|████████  | 8/10 [02:12<00:22, 11.14s/it]

[########################################] | 100% Completed |  6.0s
[########################################] | 100% Completed |  6.1s
[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.4s
[########################################] | 100% Completed |  6.5s
[########################################] | 100% Completed |  6.6s


 90%|█████████ | 9/10 [02:19<00:09,  9.81s/it]

[########################################] | 100% Completed |  5.7s
[########################################] | 100% Completed |  5.8s
[########################################] | 100% Completed |  5.9s
[########################################] | 100% Completed |  6.1s
[########################################] | 100% Completed |  6.2s
[########################################] | 100% Completed |  6.3s
[########################################] | 100% Completed |  6.4s


100%|██████████| 10/10 [02:26<00:00, 14.60s/it]
